In [35]:
import pickle
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn

t_file = open('data/train_data.pkl', 'rb')
train_data = pickle.load(t_file)
v_file = open('data/verify_data.pkl', 'rb')
verify_data = pickle.load(v_file)

users_df = pd.read_csv('data/users.csv')
items_df = pd.read_csv('data/items.csv')

In [6]:
emb_scale = 1024
batch_size = 64
len_users = len(users_df)
len_items = len(items_df)

In [8]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.feature = data[:,0:2]
        self.label = data[:,2]
        pass
    def __getitem__(self, idx):
        return self.feature[idx], self.label[idx]
    def __len__(self):
        return len(self.feature)
    pass

train_dataset = paddle.io.DataLoader(Dataset(train_data),
                               batch_size=batch_size,
                               shuffle=True)
verify_dataset = paddle.io.DataLoader(Dataset(verify_data),
                               batch_size=batch_size,
                               shuffle=True)                               

In [40]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users, emb_scale)
        self.items_emb = nn.Embedding(len_items, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        return x
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()

In [56]:
dssm.train()
optim = paddle.optimizer.Adam(parameters=dssm.parameters())
for epoch in range(5):
    for batch_id, data in enumerate(train_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')
        y_data = paddle.cast(y_data, dtype='float32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, y_data)
        # acc = paddle.metric.accuracy(y_pred, y_data)
        loss.backward()

        print('epoch:{}, batch_id:{}, loss:{}'.format(epoch, batch_id, loss))
        optim.step()
        optim.clear_grad()

epoch:0, batch_id:0, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.27331084])
epoch:0, batch_id:1, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.38305154])
epoch:0, batch_id:2, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.27104890])
epoch:0, batch_id:3, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.25922689])
epoch:0, batch_id:4, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.25106493])
epoch:0, batch_id:5, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.26464620])
epoch:0, batch_id:6, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.25041193])
epoch:0, batch_id:7, loss:Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.24626681])
epoch:0, batch_id:8, loss:Tensor(shape=[1], dtyp

ValueError: (InvalidArgument) Variable value (input) of OP(fluid.layers.embedding) expected >= 0 and < 12118, but got 12118. Please check input value.
  [Hint: Expected ids[i] < row_number, but received ids[i]:12118 >= row_number:12118.] (at C:\home\workspace\Paddle_release\paddle\phi\kernels\cpu\embedding_kernel.cc:63)
  [operator < lookup_table_v2 > error]

In [30]:
data

[Tensor(shape=[64, 2], dtype=float64, place=Place(cpu), stop_gradient=True,
        [[1146. , 17031.],
         [4950. , 10555.],
         [4979. , 5716. ],
         [2431. , 3671. ],
         [7472. , 7562. ],
         [4921. , 3812. ],
         [9802. , 1470. ],
         [343.  , 19488.],
         [2243. , 20133.],
         [2448. , 5251. ],
         [4280. , 19240.],
         [1575. , 9388. ],
         [2746. , 17934.],
         [5900. , 11502.],
         [8494. , 5323. ],
         [1879. , 8074. ],
         [7569. , 5612. ],
         [543.  , 22135.],
         [6790. , 6787. ],
         [8210. , 10629.],
         [10980., 18532.],
         [10521., 18770.],
         [6454. , 5223. ],
         [539.  , 4084. ],
         [5959. , 18073.],
         [1562. , 9796. ],
         [10087., 11181.],
         [7406. , 8037. ],
         [8942. , 21811.],
         [11180., 14558.],
         [3318. , 6478. ],
         [6565. , 18131.],
         [10693., 13031.],
         [12070., 1148. ],
      

In [51]:
y_data

Tensor(shape=[64], dtype=float64, place=Place(cpu), stop_gradient=True,
       [1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 1.])

In [45]:
y_pred

Tensor(shape=[64], dtype=float32, place=Place(cpu), stop_gradient=False,
       [-0.03624624,  0.09251690, -0.02924162, -0.03696593,  0.04947633,
        -0.04739025,  0.02739131,  0.06242286, -0.05397279,  0.09485026,
         0.12078779, -0.02650407, -0.01777668,  0.05751164,  0.07002698,
        -0.07981928,  0.08102811, -0.07164367, -0.06135988, -0.00247602,
         0.10192709,  0.07498968, -0.11466093, -0.03832404,  0.08107851,
        -0.00783026, -0.02105457,  0.04965549, -0.03060984, -0.01571941,
         0.04017977, -0.07176577, -0.01948790,  0.01310847,  0.07121481,
        -0.00315862, -0.02790038,  0.10602467, -0.15124713, -0.10561380,
         0.04842491, -0.13094470, -0.00558484, -0.02209545, -0.04983817,
        -0.09771534,  0.08581641,  0.13174142, -0.01187345,  0.10064415,
         0.16601695, -0.03741694, -0.00030887, -0.05058313,  0.00818057,
         0.06957771,  0.01286996,  0.01521156, -0.03210000,  0.10589895,
        -0.04387652,  0.04937097,  0.07548474,  0.0

In [ ]:
y_

In [49]:
paddle.nn.functional.mse_loss(y_pred, y_data)

ValueError: (InvalidArgument) The type of data we are trying to retrieve does not match the type of data currently contained in the container.
  [Hint: Expected dtype() == paddle::experimental::CppTypeToDataType<T>::Type(), but received dtype():5 != paddle::experimental::CppTypeToDataType<T>::Type():12.] (at C:\home\workspace\Paddle_release\paddle\phi\core\dense_tensor.cc:137)
  [operator < elementwise_sub > error]